In [27]:
import cv2                       # Imports OpenCV for video capture and image processing
import mediapipe as mp           # Imports MediaPipe for hand detection and landmark estimation
import numpy as np               # Imports NumPy for array and mathematical operations
import math                      # Imports math for trigonometric and square root operations
from collections import deque, Counter  # Provides data structures for smoothing and frequency counting

# Initialize the MediaPipe Hands solution with detection and tracking confidence levels
mp_hands = mp.solutions.hands
mp_draw = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    min_detection_confidence=0.5,   # Minimum confidence threshold for initial hand detection
    min_tracking_confidence=0.5     # Minimum confidence threshold for landmark tracking
)

# Define the landmark indices for each finger in MediaPipe Hands: MCP, PIP, DIP, TIP
FINGER_LANDMARKS = {
    "thumb":  [1, 2, 3, 4],     # Landmark indices for the thumb
    "index":  [5, 6, 7, 8],     # Landmark indices for the index finger
    "middle": [9, 10, 11, 12],  # Landmark indices for the middle finger
    "ring":   [13, 14, 15, 16], # Landmark indices for the ring finger
    "pinky":  [17, 18, 19, 20]  # Landmark indices for the pinky
}

def get_landmark_coords(hand_landmarks, index):
    """
    Returns the (x, y, z) coordinates of a specific landmark, given its index.
    """
    landmark = hand_landmarks.landmark[index]
    return np.array([landmark.x, landmark.y, landmark.z])

def vector(a, b):
    """
    Constructs a 3D vector going from point a to point b.
    """
    return b - a

def angle_between_points(a, b, c):
    """
    Calculates the angle formed at point b by the lines (a->b) and (b->c).
    Returns this angle in degrees.
    """
    ab = vector(b, a)
    cb = vector(b, c)
    dot_prod = np.dot(ab, cb)
    mag_ab = np.linalg.norm(ab)
    mag_cb = np.linalg.norm(cb)
    
    if mag_ab * mag_cb == 0:
        return 0.0  # Avoid division by zero
    
    cosine_angle = np.clip(dot_prod / (mag_ab * mag_cb), -1.0, 1.0)
    angle = np.degrees(np.arccos(cosine_angle))
    return angle

def is_finger_extended(hand_landmarks, finger_name, extension_threshold=160):
    """
    Determines if a particular finger is extended by measuring the angle at its PIP joint.
    If the angle is less than the extension_threshold (160 by default), we consider the finger extended.
    """
    indices = FINGER_LANDMARKS[finger_name]
    mcp_idx, pip_idx, dip_idx, tip_idx = indices
    
    mcp = get_landmark_coords(hand_landmarks, mcp_idx)
    pip = get_landmark_coords(hand_landmarks, pip_idx)
    tip = get_landmark_coords(hand_landmarks, tip_idx)

    angle_pip = angle_between_points(mcp, pip, tip)
    return angle_pip > extension_threshold

def detect_gesture(hand_landmarks):
    """
    Classifies the hand gesture using reversed logic:
      - "Pointing": Only the index finger is extended
      - "Neutral": No finger is extended
      - "Holding": Everything else
    """
    thumb_ext  = is_finger_extended(hand_landmarks, "thumb")
    index_ext  = is_finger_extended(hand_landmarks, "index")
    middle_ext = is_finger_extended(hand_landmarks, "middle")
    ring_ext   = is_finger_extended(hand_landmarks, "ring")
    pinky_ext  = is_finger_extended(hand_landmarks, "pinky")

    extended_fingers = sum([thumb_ext, index_ext, middle_ext, ring_ext, pinky_ext])

    # 1. Pointing: Only the index finger is extended
    if (index_ext
        and not thumb_ext
        and not middle_ext
        and not ring_ext
        and not pinky_ext):
        return "Pointing"

    # 2. Neutral: No finger is extended
    if extended_fingers == 0:
        return "Holding"

    # 3. Holding: Any other scenario
    return "Neutral"




### **🔹 Key Features**  
✅ **Uses MediaPipe Hands** for detecting and tracking **21 hand landmarks**.  
✅ **Determines finger positions** by analyzing joint angles.  
✅ **Classifies gestures dynamically** into predefined categories.  

---

### **🔹 How It Works**  

1. **Hand Detection (`mp_hands`)**  
   - Detects **21 hand landmarks** per hand.
   - Tracks **fingers and joints** over time.

2. **Finger Landmark Definitions (`FINGER_LANDMARKS`)**  
   - Stores the key landmarks of each finger:  
     - **MCP** → Base joint  
     - **PIP** → Middle joint  
     - **DIP** → Near fingertip joint  
     - **TIP** → Fingertip  

3. **Extract Landmark Coordinates (`get_landmark_coords()`)**  
   - Retrieves the **(x, y, z)** coordinates of any hand landmark.

4. **Calculate Finger Orientation (`vector()`, `angle_between_points()`)**  
   - Computes the **angle between joints** to check finger extension.

5. **Finger Extension Detection (`is_finger_extended()`)**  
   - A **finger is extended** if its **PIP joint angle** exceeds **160°**.

6. **Gesture Classification (`detect_gesture()`)**  
   - **"Pointing"** → Only the index finger is extended.  
   - **"Neutral"** → No fingers are extended.  
   - **"Holding"** → Any other combination of fingers extended.  

In [28]:
# GESTURE_HISTORY_SIZE determines how many recent frames we keep to smooth out the classification
GESTURE_HISTORY_SIZE = 10

# Create a deque (double-ended queue) to store the most recent gesture results
gesture_history = deque(maxlen=GESTURE_HISTORY_SIZE)

# Start capturing video from the default webcam
cap = cv2.VideoCapture(0)

while cap.isOpened():                  # Continue until the webcam is closed or an error occurs
    success, frame = cap.read()        # Read a frame from the camera
    if not success:                    # If the frame is not read successfully, break the loop
        break

    frame = cv2.flip(frame, 1)         # Flip the frame horizontally for a mirror-like view
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert the BGR frame to RGB for MediaPipe
    results = hands.process(rgb_frame)                  # Process the frame to detect hands

    current_gesture = "No Hand"        # Default gesture if no hand is detected
    if results.multi_hand_landmarks:   # If there's at least one hand detected
        # We only handle the first detected hand in this example
        hand_landmarks = results.multi_hand_landmarks[0]

        # Draw the detected hand landmarks on the frame
        mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        # Determine the gesture using the reversed logic
        current_gesture = detect_gesture(hand_landmarks)

    # Append the current gesture to our history for smoothing
    gesture_history.append(current_gesture)
    # Find the most common gesture in the recent history to reduce classification flicker
    most_common_gesture = Counter(gesture_history).most_common(1)[0][0]

    # Show the smoothed gesture classification on the screen
    cv2.putText(
        frame,
        f"Gesture: {most_common_gesture}",
        (50, 50),
        cv2.FONT_HERSHEY_SIMPLEX,
        1,
        (0, 255, 0),
        2
    )

    # If there's at least one hand, display the number of extended fingers for debugging
    if results.multi_hand_landmarks:
        sum_extended = 0
        for name in FINGER_LANDMARKS.keys():
            ext = is_finger_extended(hand_landmarks, name)
            sum_extended += int(ext)
        cv2.putText(
            frame,
            f"Extended Fingers: {sum_extended}",
            (50, 100),
            cv2.FONT_HERSHEY_SIMPLEX,
            1,
            (0, 255, 255),
            2
        )

    # Display the augmented frame in a new window
    cv2.imshow("Reversed Logic Hand Gesture Detection", frame)

    # If 'q' is pressed, exit the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close the display window
cap.release()
cv2.destroyAllWindows()


### **🔹 Key Features**  
✅ **Real-time hand gesture detection using OpenCV and MediaPipe**.  
✅ **Smooths gesture classification using a sliding window (`deque`)**.  
✅ **Displays gesture type and number of extended fingers on the screen**.  

---

### **🔹 How It Works**  

1. **Gesture Smoothing with History (`gesture_history`)**  
   - Uses a **deque (double-ended queue)** to store the last **10 detected gestures**.
   - **Finds the most common gesture** in recent frames to reduce flickering.

2. **Webcam Frame Processing (`cv2.VideoCapture(0)`)**  
   - Captures live video from the **default webcam**.
   - **Flips the frame horizontally** for a natural, mirror-like view.
   - **Converts the frame from BGR to RGB** for MediaPipe hand detection.

3. **Hand Detection & Gesture Classification (`detect_gesture()`)**  
   - Uses **MediaPipe Hands** to detect hand landmarks.
   - If a hand is detected, **draws landmarks** on the frame.
   - **Classifies the detected gesture** based on finger positions.

4. **Counting Extended Fingers (`is_finger_extended()`)**  
   - Iterates through all **five fingers** and checks if they are extended.
   - Displays the **total number of extended fingers** on the screen.

5. **Displaying Information on Screen (`cv2.putText()`)**  
   - Shows the **most common gesture** from the last 10 frames.
   - Shows **number of extended fingers** if a hand is detected.

6. **Live Video Feed (`cv2.imshow()`)**  
   - Continuously displays the processed video feed with **gesture classification overlays**.
   - Press **'q' to exit** and close the webcam feed.

7. **Cleanup (`cap.release()` & `cv2.destroyAllWindows()`)**  
   - Properly releases the webcam and **closes all OpenCV windows** when exiting.

In [5]:

from roboflow import Roboflow
rf = Roboflow(api_key="JgNlmmfB1eTF3aW7fXFJ")
project = rf.workspace("product-detection-tcbq8").project("my-first-project-qdnml")
version = project.version(2)
dataset = version.download("yolov8")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to My-First-Project-2 in yolov8:: 100%|██████████| 654/654 [00:00<00:00, 1090.26it/s]


### **🔹 Key Features**  
✅ **Uses the Roboflow API** to download a YOLOv8 dataset.  
✅ **Connects to a specific project inside a workspace**.  
✅ **Downloads the dataset in the correct format for YOLOv8 training**.  

---

### **🔹 How It Works**  

1. **Initialize the Roboflow Client (`Roboflow(api_key)`)**  
   - Requires a **Roboflow API key** (`JgNlmmfB1eTF3aW7fXFJ` in this example).  
   - Connects to the **Roboflow cloud platform** to access datasets.  

2. **Select the Project (`rf.workspace().project()`)**  
   - **Workspace:** `"product-detection-tcbq8"` (group of related projects).  
   - **Project Name:** `"my-first-project-qdnml"` (specific dataset being used).  

3. **Specify the Dataset Version (`project.version(2)`)**  
   - Chooses **version 2** of the dataset (versions track different updates of a dataset).  

4. **Download the Dataset (`version.download("yolov8")`)**  
   - Fetches the dataset in **YOLOv8 format**, making it **ready for training**.  
   - Automatically downloads and **extracts labeled images** to your local directory.  

---

### **🔹 Notes**  
- The **API key should be kept secure**—avoid sharing publicly.  
- Ensure the **Roboflow workspace & project names are correct** to avoid errors.  
- The dataset is downloaded into the **current working directory**, so check the output folder after running this script.

In [ ]:
from ultralytics import YOLO

# Load a YOLOv8 model (choose 'n' for nano version)
model = YOLO("yolov8n.pt")

# Train on your dataset
model.train(data=r"C:\Users\Adonis\OneDrive\Desktop\DataScience\Projects\MotionDetection\RoboFLow\My-First-Project-2\data.yaml", epochs=50)



New https://pypi.org/project/ultralytics/8.3.78 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.74  Python-3.11.11 torch-2.6.0+cpu CPU (AMD Ryzen 5 5600H with Radeon Graphics)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=C:\Users\Adonis\OneDrive\Desktop\DataScience\Projects\MotionDetection\My-First-Project-2\data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agno

RuntimeError: Dataset 'C://Users/Adonis/OneDrive/Desktop/DataScience/Projects/MotionDetection/My-First-Project-2/data.yaml' error  
Dataset 'C://Users/Adonis/OneDrive/Desktop/DataScience/Projects/MotionDetection/My-First-Project-2/data.yaml' images not found , missing path 'C:\Users\Adonis\OneDrive\Desktop\DataScience\Projects\MotionDetection\My-First-Project-2\valid\images'
Note dataset download directory is 'C:\Users\Adonis\OneDrive\Desktop\DataScience\Projects\SuperMarket Traffic\datasets'. You can update this in 'C:\Users\Adonis\AppData\Roaming\Ultralytics\settings.json'

### **🔹 Key Features**  
✅ **Loads YOLOv8 (Nano version) for object detection**.  
✅ **Trains the model on a custom dataset from Roboflow**.  
✅ **Runs for 50 epochs to optimize model performance**.  

---

### **🔹 How It Works**  

1. **Import YOLO from Ultralytics (`from ultralytics import YOLO`)**  
   - Loads the **YOLOv8 framework**, which supports training, validation, and inference.  

2. **Load the YOLOv8 Model (`YOLO("yolov8n.pt")`)**  
   - Uses the **Nano version (`yolov8n.pt`)**, which is lightweight and fast.  
   - Suitable for **real-time applications** on devices with limited computational power.  

3. **Train the Model (`model.train()`)**  
   - **Dataset File:** `data.yaml` → Contains dataset paths, class labels, and configurations.  
   - **Training Directory:** `C:\Users\Adonis\OneDrive\Desktop\DataScience\Projects\MotionDetection\RoboFLow\My-First-Project-2\`  
   - **Epochs:** `50` → The number of times the model sees the dataset to learn features.  
   - **During training**, YOLO:
     - Loads labeled images from the dataset.  
     - Performs **augmentation** to improve generalization.  
     - **Optimizes weights** for object detection.  

---

### **🔹 Notes**  
- Ensure that **`data.yaml` is correctly formatted** with dataset paths and class names.  
- The number of epochs (`epochs=50`) can be **increased for better accuracy** or **decreased for faster training**.  
- After training, the **best model weights** will be saved (e.g., `runs/train/exp/weights/best.pt`).  
- This model can later be used for **real-time detection** in videos or images.

In [12]:
import cv2
from ultralytics import YOLO

# Load the trained YOLOv8 model
model = YOLO(r"C:\Users\Adonis\OneDrive\Desktop\DataScience\Projects\MotionDetection\detect\train11\weights\best.pt")

# Initialize webcam (0 = default camera)
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Inference
    results = model.predict(frame, conf=0.8)  # set your preferred confidence threshold

    # Copy the frame so we can draw on it
    annotated_frame = frame.copy()

    # results is a list, but there's only one item for a single frame
    detections = results[0]

    # Loop through each detection
    for box in detections.boxes:
        # box.xyxy[0] -> [x1, y1, x2, y2]
        x1, y1, x2, y2 = box.xyxy[0]
        # Convert to integer
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

        # box.cls[0] -> class index, box.conf[0] -> confidence
        cls_id = int(box.cls[0])
        conf = float(box.conf[0])

        # Get class name using the model's names dictionary
        class_name = model.names[cls_id]

        # Draw bounding box
        color = (255, 0, 0)  # BGR: Blue
        cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), color, 2)

        # Create label text
        label = f"{class_name} {conf:.2f}"

        # Draw label background box for better text visibility
        (text_width, text_height), baseline = cv2.getTextSize(
            label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2
        )
        cv2.rectangle(
            annotated_frame, 
            (x1, y1 - text_height - 10), 
            (x1 + text_width, y1), 
            color, 
            -1
        )
        
        # Put label text above bounding box
        cv2.putText(
            annotated_frame, 
            label, 
            (x1, y1 - 5), 
            cv2.FONT_HERSHEY_SIMPLEX, 
            0.5, 
            (255, 255, 255),  # White text
            2
        )

    # Show the annotated frame
    cv2.imshow("YOLOv8 Real-Time Detection", annotated_frame)

    # Press ESC key to quit
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()



0: 480x640 (no detections), 151.4ms
Speed: 8.7ms preprocess, 151.4ms inference, 8.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 144.7ms
Speed: 4.5ms preprocess, 144.7ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 78.4ms
Speed: 3.0ms preprocess, 78.4ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 76.2ms
Speed: 2.0ms preprocess, 76.2ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 67.9ms
Speed: 1.0ms preprocess, 67.9ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 69.5ms
Speed: 2.5ms preprocess, 69.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 69.3ms
Speed: 1.5ms preprocess, 69.3ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 62.1ms
Speed: 2.0ms preprocess, 62.1

### **🔹 Key Features**  
✅ **Loads a trained YOLOv8 model for real-time object detection**.  
✅ **Captures live video from the webcam and processes each frame**.  
✅ **Draws bounding boxes and labels around detected objects**.  
✅ **Displays confidence scores to indicate detection certainty**.  

---

### **🔹 How It Works**  

1. **Load the Trained YOLOv8 Model (`YOLO(best.pt)`)**  
   - Uses a **custom-trained YOLOv8 model** (`best.pt`) to recognize objects.
   - The model is located at:  
     ```
     C:\Users\Adonis\OneDrive\Desktop\DataScience\Projects\MotionDetection\detect\train11\weights\best.pt
     ```
   - **Ensures the model is trained and available before running detection.**

2. **Capture Live Video (`cv2.VideoCapture(0)`)**  
   - Uses the **default webcam** (`0`) for real-time detection.
   - Reads frames in a continuous loop.

3. **Run YOLOv8 Inference (`model.predict()`)**  
   - Processes each frame and predicts objects with **confidence ≥ 0.8**.
   - `results[0]` contains detected bounding boxes, class IDs, and confidence scores.

4. **Extract & Draw Bounding Boxes**  
   - Extracts **coordinates (`x1, y1, x2, y2`)** for each detected object.  
   - Draws a **blue rectangle (`(255, 0, 0)`)** around the detected object.  
   - Fetches the **class name from `model.names[]`** and displays it.

5. **Overlay Detection Labels**  
   - Displays the **object name + confidence score** above the bounding box.
   - Uses a **background box** behind the text to improve visibility.

6. **Display the Processed Frame (`cv2.imshow()`)**  
   - Shows the **annotated video feed** in a new window.
   - **Updates dynamically** as new frames are processed.

7. **Exit on ESC Key (`cv2.waitKey(1)`)**  
   - Press **ESC (`27`)** to close the detection window.
   - Ensures **graceful shutdown** by releasing the webcam and closing OpenCV windows.

---

### **🔹 Notes**  
- The **YOLO model should be trained** and tested before use.  
- The confidence threshold (`conf=0.8`) can be **adjusted** for better results.  
- The script runs **in real-time**, so performance depends on **hardware (CPU/GPU)**.  
- Bounding boxes and labels are drawn with **OpenCV (`cv2.rectangle()` & `cv2.putText()`)**.

In [ ]:
import sqlite3

# Function to fetch product details by name
def get_product_details(product_name):
    conn = sqlite3.connect("products.db")
    cursor = conn.cursor()

    # Query database for the product
    cursor.execute("SELECT * FROM products WHERE name = ?", (product_name,))
    product = cursor.fetchone()

    conn.close()
    return product  # Returns None if no product is found


### **🔹 Key Features**  
✅ **Connects to a SQLite database (`products.db`)**.  
✅ **Fetches product details based on the product name**.  
✅ **Returns all stored product information (or `None` if not found)**.  

---

### **🔹 How It Works**  

1. **Establish a Database Connection (`sqlite3.connect()`)**  
   - Connects to **`products.db`**, which stores product details.  
   - Allows SQL queries to be executed on the database.  

2. **Execute a SQL Query (`cursor.execute()`)**  
   - Runs a **parameterized query** (`?` placeholder) to prevent SQL injection.  
   - Searches for a product where the **name matches the input (`product_name`)**.  

3. **Retrieve the Product Data (`cursor.fetchone()`)**  
   - Returns **the first matching row** as a tuple.  
   - If **no match is found**, it returns `None`.  

4. **Close the Database Connection (`conn.close()`)**  
   - Ensures the database connection is properly **closed after the query**.  

---

### **🔹 Notes**  
- The function assumes that the **product name is unique** in the database.  
- If multiple products have the same name, **only the first match** will be returned.  
- The function can be modified to return **all matches** using `cursor.fetchall()`.  
- The database file (`products.db`) **must exist** and contain the `products` table for this function to work correctly.  

In [9]:
import database_queries  # Import function to fetch product details

# Load YOLOv8 model
model = YOLO("best.pt")  # Replace with your trained YOLOv8 model path

# Capture an image (or use a test image)
image_path = r"C:\Users\Adonis\OneDrive\Desktop\DataScience\Projects\MotionDetection\StreamLit\Images\Teriaq-Intense.jpeg"
image = cv2.imread(image_path)

# Run YOLO prediction
results = model(image)

# Ensure detections exist
if results and len(results[0].boxes) > 0:  
    # Get the class ID of the detected object
    detected_class_id = int(results[0].boxes.cls[0])  

    # Get the corresponding label name
    detected_label = model.names[detected_class_id]  

    print(f"Detected Product: {detected_label}")

    # Query database for product details
    product_info = database_queries.get_product_details(detected_label)

    if product_info:
        print("\nProduct Details Found:")
        print(f"ID: {product_info[0]}")
        print(f"Name: {product_info[1]}")
        print(f"Category: {product_info[2]}")
        print(f"Genre: {product_info[3]}")
        print(f"Price: ${product_info[4]}")
        print(f"Description: {product_info[5]}")
        print(f"Image Path: {product_info[6]}")
    else:
        print("\n❌ Product not found in database!")
else:
    print("\n❌ No product detected!")



0: 640x544 1 Teriaq-Intense, 198.7ms
Speed: 4.5ms preprocess, 198.7ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 544)
Detected Product: Teriaq-Intense

Product Details Found:
ID: 1
Name: Teriaq-Intense
Category: Perfume
Genre: Sweet
Price: $180.0
Description: A rich and captivating sweet fragrance with deep notes of vanilla and caramel, leaving a long-lasting impression.
Image Path: C:\Users\Adonis\OneDrive\Desktop\DataScience\Projects\MotionDetection\StreamLit\Images\Teriaq-Intense.jpeg


### **🔹 Key Features**  
✅ **Loads a trained YOLOv8 model for object detection**.  
✅ **Detects objects in a given image** using YOLOv8.  
✅ **Extracts the detected object label and queries a SQLite database for details**.  
✅ **Displays product details (ID, name, category, genre, price, description, and image path)**.  

---

### **🔹 How It Works**  

1. **Import the `database_queries` module**  
   - Loads the function `get_product_details()`, which fetches product details from the database.

2. **Load the YOLOv8 Model (`YOLO("best.pt")`)**  
   - Loads a **custom-trained YOLOv8 model** to detect objects.  
   - The model is stored in `best.pt`, which contains learned weights from training.

3. **Read the Image (`cv2.imread()`)**  
   - Loads an image from a specified path (`Teriaq-Intense.jpeg`).  
   - The image is used as input for YOLOv8 object detection.

4. **Run Object Detection (`model(image)`)**  
   - YOLO processes the image and detects objects.  
   - Returns **bounding boxes, class IDs, and confidence scores**.

5. **Extract the Detected Object's Class Label**  
   - **`boxes.cls[0]`** retrieves the **class index** of the detected object.  
   - **`model.names[class_id]`** maps the class index to a **human-readable label**.

6. **Query the Database (`database_queries.get_product_details()`)**  
   - Searches the `products.db` database for the detected product name.  
   - Returns product details if the product is found.

7. **Display Product Details**  
   - If the product exists in the database, prints:  
     - **ID, Name, Category, Genre, Price, Description, and Image Path**.  
   - If not found, displays: **"❌ Product not found in database!"**.

8. **Handle Cases When No Object is Detected**  
   - If YOLOv8 **detects nothing**, the script prints `"❌ No product detected!"`.  
   - Prevents crashes when no valid detection is made.

---

### **🔹 Notes**  
- The **YOLOv8 model (`best.pt`) should be trained** on the dataset containing product images.  
- Ensure the **image file exists at the given path** before running detection.  
- The **SQLite database (`products.db`) should contain relevant product information** for proper retrieval.  
- If detection is inaccurate, consider **lowering the confidence threshold** when running `model(image)`.  

In [1]:
import sqlite3

conn = sqlite3.connect("products.db")
cursor = conn.cursor()

# Check database integrity
cursor.execute("PRAGMA integrity_check;")
result = cursor.fetchone()

print("Database Integrity Check:", result[0])

conn.close()


Database Integrity Check: ok


### **🔹 Key Features**  
✅ **Connects to the SQLite database (`products.db`)**.  
✅ **Runs an integrity check to detect database corruption**.  
✅ **Prints the status of the database (`ok` if no corruption is found)**.  
✅ **Closes the database connection after the check**.  

---

### **🔹 How It Works**  

1. **Establish a Database Connection (`sqlite3.connect()`)**  
   - Opens a connection to the SQLite database **`products.db`**.  
   - Allows SQL commands to be executed on the database.

2. **Execute an Integrity Check (`PRAGMA integrity_check;`)**  
   - **PRAGMA integrity_check** is a built-in SQLite command that:  
     - Scans the database for corruption.  
     - Verifies that **all tables and indexes are properly structured**.  
     - Detects **missing or broken references**.  

3. **Retrieve the Integrity Check Result (`cursor.fetchone()`)**  
   - Fetches the **output** of the integrity check:  
     - If **`ok`**, the database is healthy.  
     - If **an error message appears**, the database is corrupted.

4. **Print the Integrity Status**  
   - Displays the **status of the database** in the console.  
   - Helps diagnose **corruption issues before running queries**.

5. **Close the Database Connection (`conn.close()`)**  
   - Ensures the connection is **properly closed** after executing the check.  

---

### **🔹 Notes**  
- If the **output is `"ok"`**, your database is **healthy and working properly**.  
- If the **output shows errors**, your database **may be corrupted** and needs to be repaired.  
- If corruption is detected, try **dumping the database using `.dump`** or **rebuilding it from a backup**.  

In [24]:
import database_queries  # Import function to fetch product details
import os

# Load YOLOv8 model
model = YOLO("best.pt")  # Replace with your trained YOLOv8 model path

# Open the webcam (0 = default camera)
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()  # Capture a frame from the webcam
    if not ret:
        break  # Exit if the frame is not captured properly

    # Run YOLO detection on the frame
    results = model(frame)

    # Ensure at least one object is detected
    if results and len(results[0].boxes) > 0:
        # Get the class ID of the detected object
        detected_class_id = int(results[0].boxes.cls[0])

        # Get the corresponding label name from YOLO's class names
        detected_label = model.names[detected_class_id]
        print(f"Detected Product: {detected_label}")

        # Query the database to get product details
        product_info = database_queries.get_product_details(detected_label)

        if product_info:
            # Extract product details from database
            product_id, name, category, genre, price, description, image_path = product_info

            # Display ALL product details as text overlay
            overlay_text = [
                f"ID: {product_id}",
                f"Name: {name}",
                f"Category: {category}",
                f"Genre: {genre}",
                f"Price: ${price}",
                f"Description: {description}",
            ]

            y_offset = 50  # Start position for text overlay
            for line in overlay_text:
                cv2.putText(frame, line, (50, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
                y_offset += 25  # Move down for each line

            # Display product image if available
            if os.path.exists(image_path):
                img = cv2.imread(image_path)  # Read the product image
                img = cv2.resize(img, (150, 150))  # Resize to fit overlay area
                frame[50:200, 400:550] = img  # Overlay the image on the video feed
        else:
            # If product is not found in the database
            cv2.putText(frame, "Product not found!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Display the real-time video feed with product details
    cv2.imshow("Product Recognition", frame)

    # Press 'q' to exit the program
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()



0: 480x640 (no detections), 66.7ms
Speed: 4.0ms preprocess, 66.7ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 71.5ms
Speed: 2.0ms preprocess, 71.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 60.4ms
Speed: 3.0ms preprocess, 60.4ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 54.5ms
Speed: 2.8ms preprocess, 54.5ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 54.9ms
Speed: 1.1ms preprocess, 54.9ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 55.1ms
Speed: 1.7ms preprocess, 55.1ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 52.1ms
Speed: 2.2ms preprocess, 52.1ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 50.8ms
Speed: 1.0ms preprocess, 50.8ms i

---

### **📄 File: `real_time_detection.py`**

### **🔹 How This Works**
1. **Captures live video from the webcam**.
2. **Runs YOLOv8 object detection** on each frame.
3. **Extracts the detected product label** from YOLO’s predictions.
4. **Searches for the product in the SQLite database**.
5. **Displays product details** (name, price, description) as an overlay on the video feed.
6. **Displays product image** (if found) on the right side of the frame.
7. **Press 'q' to exit** the live detection.




---

### **💡 Next Steps**
✅ **Want to integrate it into a Streamlit web app?** 🖥  
✅ **Need help training YOLOv8 on your custom product dataset?** 📦  
✅ **Want to add voice-based product info (text-to-speech)?** 🔊  


In [32]:
import cv2
import mediapipe as mp
import numpy as np
from ultralytics import YOLO
import database_queries
import os
from collections import deque, Counter

# Initialize MediaPipe drawing utilities
mp_draw = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

# Load YOLOv8 model
model = YOLO("best.pt")

# Gesture smoothing buffer
GESTURE_HISTORY_SIZE = 10
gesture_history = deque(maxlen=GESTURE_HISTORY_SIZE)

# Start video capture
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)  # Use imported 'hands' model

    current_gesture = "No Hand"
    
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Draw hand landmarks and connections
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                                   mp_draw.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=4),
                                   mp_draw.DrawingSpec(color=(0, 0, 255), thickness=2))

            current_gesture = detect_gesture(hand_landmarks)  # Detect gesture using existing function

    gesture_history.append(current_gesture)
    most_common_gesture = Counter(gesture_history).most_common(1)[0][0]

    # Display detected gesture
    cv2.putText(frame, f"Gesture: {most_common_gesture}", (50, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Run YOLO only if "Pointing" or "Holding"
    if most_common_gesture in ["Pointing", "Holding"]:
        results = model.predict(frame, conf=0.7)
        if results and len(results[0].boxes) > 0:
            for box in results[0].boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                cls_id, conf = int(box.cls[0]), float(box.conf[0])
                detected_label = model.names[cls_id]

                # Fetch product details
                product_info = database_queries.get_product_details(detected_label)

                if product_info:
                    product_id, name, category, genre, price, description, image_path = product_info
                    details = [f"ID: {product_id}", f"Name: {name}", f"Category: {category}",
                               f"Genre: {genre}", f"Price: ${price}", f"Desc: {description}"]
                    y_offset = 80
                    for detail in details:
                        cv2.putText(frame, detail, (50, y_offset),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
                        y_offset += 25

                    # Display product image
                    if os.path.exists(image_path):
                        img = cv2.imread(image_path)
                        img = cv2.resize(img, (150, 150))
                        frame[50:200, 400:550] = img
                else:
                    cv2.putText(frame, "Product not found!", (50, 80),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Show the video
    cv2.imshow("Gesture-Based Product Recognition", frame)

    # Exit when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



0: 480x640 (no detections), 51.7ms
Speed: 1.0ms preprocess, 51.7ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 56.5ms
Speed: 0.5ms preprocess, 56.5ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 52.5ms
Speed: 1.0ms preprocess, 52.5ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Afnan, 54.4ms
Speed: 1.0ms preprocess, 54.4ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Afnan, 59.2ms
Speed: 1.5ms preprocess, 59.2ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Afnan, 1 Teriaq-Intense, 58.3ms
Speed: 1.0ms preprocess, 58.3ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Afnan, 55.6ms
Speed: 1.0ms preprocess, 55.6ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Afnan, 1 Teriaq-Intense, 57.7ms
Speed: 1.1ms preprocess, 57.7ms infer

<function dir>